In [3]:
from datasets import load_dataset

# 加载 WMT14 英语-法语数据集
dataset = load_dataset("wmt14", "fr-en", split="train[:5000]")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

train-00000-of-00030.parquet:   0%|          | 0.00/252M [00:00<?, ?B/s]

train-00001-of-00030.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/536k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40836715 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

Dataset({
    features: ['translation'],
    num_rows: 5000
})


In [4]:
print(dataset[0])

{'translation': {'en': 'Resumption of the session', 'fr': 'Reprise de la session'}}


In [6]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors

# 初始化一个 BPE 模型
tokenizer = Tokenizer(models.BPE())

# 设置预分词器（按空格分词）
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 设置解码器
tokenizer.decoder = decoders.ByteLevel()

# 训练分词器
trainer = trainers.BpeTrainer(
    vocab_size=30000,  # 词汇表大小
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
)

# 准备训练数据
training_data = []
for example in dataset:
    training_data.append(example['translation']['en'])
    training_data.append(example['translation']['fr'])

# 训练分词器
tokenizer.train_from_iterator(training_data, trainer=trainer)

# 保存分词器
tokenizer.save("bpe_tokenizer.json")

In [7]:
# 加载训练好的分词器
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

# 测试分词器
sample_text = "This is a test sentence."
encoded = tokenizer.encode(sample_text)
print("Tokens:", encoded.tokens)
print("IDs:", encoded.ids)

Tokens: ['This', 'is', 'a', 'test', 'sentence', '.']
IDs: [544, 115, 56, 3304, 14022, 13]


In [8]:
from torch.utils.data import Dataset

class LangPairDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]['translation']
        src_text = example['en']
        tgt_text = example['fr']

        # 对源语言和目标语言进行分词
        src_encoded = self.tokenizer.encode(src_text)
        tgt_encoded = self.tokenizer.encode(tgt_text)

        # 截断和填充
        src_ids = src_encoded.ids[:self.max_length]
        tgt_ids = tgt_encoded.ids[:self.max_length]
        src_ids = src_ids + [0] * (self.max_length - len(src_ids))  # 填充
        tgt_ids = tgt_ids + [0] * (self.max_length - len(tgt_ids))  # 填充

        return {
            "input_ids": src_ids,
            "labels": tgt_ids
        }

# 创建数据集
lang_pair_dataset = LangPairDataset(dataset, tokenizer)
print(lang_pair_dataset[0])  # 查看一条样本

{'input_ids': [21971, 138, 120, 2967, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [19948, 118, 126, 2967, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [9]:
from torch.utils.data import DataLoader

# 创建 DataLoader
dataloader = DataLoader(lang_pair_dataset, batch_size=8, shuffle=True)

# 查看一个批次的数据
for batch in dataloader:
    print(batch)
    break

{'input_ids': [tensor([ 544, 1033,  398, 9318,  398, 5240, 2467, 1113]), tensor([ 290,  120,  540,   11, 1082,  299,  943,  115]), tensor([4669,  635,  136,  367, 1394,   11,  517,   11]), tensor([15246,  2300,   716,   224, 22662,   120,   115,   114]), tensor([2272,  138,  223, 1484,  146,  473,  181,  816]), tensor([ 112,  120, 1689,   11, 8368, 4195,  450,   11]), tensor([   12,   709,  4707,  3979,    11, 10340,   411,    56]), tensor([ 3544,  1613,    13,  5693, 23184,   197,   258,  2176]), tensor([  120,  3856,     0,   114, 21309,  1147,   168,   138]), tensor([2639,  314,    0,  120,  256,  254, 8042,   56]), tensor([ 138,  120,    0,  198,   11, 6433,  498, 7289]), tensor([ 120, 9732,    0,   11, 7153, 5436,  120, 3213]), tensor([ 321,  138,    0,  146,  136, 2522,  485,  114]), tensor([1733, 1483,    0,  743, 3512,  299, 2310,  120]), tensor([ 114,  314,    0,  224, 3874, 1222,  136, 1241]), tensor([ 223,  120,    0,  114,   11, 1985,  168,   13]), tensor([ 1544,   989,    

In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.9 MB/s eta 0:00:00
